In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
# List of chosen words
chosen_words = ["game", "study", "work", "relax"]

# Find and display similar words for each of the chosen words
for word in chosen_words:
    if word in glove_wiki_vectors:
        print(f"Similar words to '{word}':")
        similar_words = glove_wiki_vectors.most_similar(word, topn=5)
        for similar_word, similarity in similar_words:
            print(f"  {similar_word}: {similarity:.4f}")
        print()
    else:
        print(f"'{word}' not found in the vocabulary!")


Similar words to 'game':
  games: 0.8645
  play: 0.8316
  season: 0.7733
  player: 0.7580
  players: 0.7288

Similar words to 'study':
  studies: 0.9033
  research: 0.8289
  researchers: 0.8094
  studying: 0.7740
  scientific: 0.7339

Similar words to 'work':
  working: 0.8774
  works: 0.8548
  well: 0.8274
  worked: 0.8128
  done: 0.8112

Similar words to 'relax':
  relaxing: 0.7026
  loosen: 0.6953
  tighten: 0.6623
  relaxed: 0.6421
  ease: 0.6058



In [7]:
...

Ellipsis

In [8]:
...

Ellipsis

In [9]:
...

Ellipsis

In [10]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
for word1, word2 in word_pairs:
    if word1 in glove_wiki_vectors and word2 in glove_wiki_vectors:
        similarity = glove_wiki_vectors.similarity(word1, word2)
        print(f"Similarity between '{word1}' and '{word2}': {similarity:.4f}")
    else:
        print(f"One or both of the words ('{word1}', '{word2}') are not in the vocabulary!")

Similarity between 'coast' and 'shore': 0.7000
Similarity between 'clothes' and 'closet': 0.5463
Similarity between 'old' and 'new': 0.6432
Similarity between 'smart' and 'intelligent': 0.7553
Similarity between 'dog' and 'cat': 0.8798
Similarity between 'tree' and 'lawyer': 0.0767


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
for word in test_words:
    if word in glove_wiki_vectors:
        print(f"'{word}' is in the vocabulary.")
    else:
        print(f"'{word}' is NOT in the vocabulary.")

'covididiot' is NOT in the vocabulary.
'fomo' is NOT in the vocabulary.
'frenemies' is in the vocabulary.
'anthropause' is NOT in the vocabulary.
'photobomb' is NOT in the vocabulary.
'selfie' is NOT in the vocabulary.
'pxg' is NOT in the vocabulary.
'pacg' is NOT in the vocabulary.
'cct' is in the vocabulary.
'escc' is in the vocabulary.


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283241

In [18]:
glove_wiki_vectors.similarity("white", "success")

0.3518238

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
print(analogy("teacher", "school", "farmer"))

teacher : school :: farmer : ?
  Analogy word     Score
0         farm  0.761160
1      farmers  0.652871
2        farms  0.649870
3        dairy  0.600245
4        mills  0.596487
5         mill  0.596371
6      farming  0.590164
7       county  0.584363
8       cattle  0.583114
9     township  0.579034


In [20]:
print(analogy("nurse", "woman", "pilot"))

nurse : woman :: pilot : ?
  Analogy word     Score
0        plane  0.672722
1          man  0.651018
2      another  0.647709
3        crash  0.629156
4          jet  0.626623
5          one  0.626047
6   apparently  0.613751
7         crew  0.613686
8      vehicle  0.610455
9     airplane  0.594764


In [21]:
print(analogy("dog", "puppy", "cat"))

dog : puppy :: cat : ?
  Analogy word     Score
0      puppies  0.686760
1       kitten  0.686680
2      kittens  0.638370
3       monkey  0.617109
4       rabbit  0.613682
5          pup  0.605465
6        tabby  0.593700
7    retriever  0.593433
8        bitch  0.581778
9        hound  0.577856


In [22]:
print(analogy("car", "road", "boat"))

car : road :: boat : ?
  Analogy word     Score
0        ferry  0.661479
1        route  0.658688
2        river  0.643877
3        boats  0.620386
4       bridge  0.617599
5       harbor  0.610785
6      harbour  0.601909
7     crossing  0.598468
8        shore  0.592136
9      sailing  0.587513


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. The analogy associates "farmer" with "farms," "dairy," "mills," etc., which are reasonable associations. However, for Nurse:Woman::Pilot:?, the result shows "man" as a top analogy word. This indicates a gender bias in the embeddings, associating nurses with women and pilots with men. Such biases reflect societal stereotypes about professions being gendered. The other two analogy are quite normal and unbiased.
2.  If embeddings are used in job recruitment software, gender or racial biases in the embeddings might cause the system to favor male candidates for STEM jobs or white candidates for leadership roles based on biased word associations.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [23]:
from sklearn.datasets import fetch_20newsgroups

In [24]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [25]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [26]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [27]:
import re
def preprocess_text(text):
    
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\s]", " ", text)
    
    doc = nlp(text)
    processed_tokens = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop
        and not token.is_punct
        and not token.like_url
        and not token.like_email
        and not token.is_digit
    ]
    return " ".join(processed_tokens)

In [28]:
df["text_pp"] = df["text"].apply(preprocess_text)

In [29]:
df.to_csv("preprocessed_data.csv", index=False)
# df = pd.read_csv("preprocessed_data.csv")

In [30]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay don t hand don t support idea ha...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create ...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat a...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,hi d like subscribe leadership magazine wond...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

1. Removal of Newline Characters and Extra Whitespace:

    Used regex (re.sub) to replace newline characters (\n) and multiple spaces with a single space.

    Justification: Newline characters and extra whitespace can disrupt tokenization and reduce the quality of text analysis. Removing them ensures cleaner input. 

2. Removal of Special Characters:

   Used regex to remove non-alphanumeric characters except spaces.


    Justification: Special characters often do not contribute meaningfully to topic modeling and can introduce noise. Cleaning them ensures focus on meaningful text

3. Tokenization:

   Split the text into smaller units (tokens) using spaCy.

    Justification: Tokenization is a fundamental step to prepare raw text for further analysis. It helps to isolate meaningful components like words or phrases.

4. Lemmatization:

    Converted each token to its base form using the lemma_ attribute.

   
    Justification: Lemmatization reduces inflected words to their root forms, enabling us to group variations of a word (e.g., "running," "runs," and "ran" → "run") into a single representation.

5. Stopword Removal:

    Removed commonly used words like "the," "is," "and," etc., using spaCy's is_stop attribute.

   
    Justification: Stopwords carry little semantic value and can introduce noise in tasks like topic modeling or classification.

6. Punctuation Removal:

    Excluded tokens identified as punctuation using the is_punct attribute.

    
    Justification: Punctuation marks do not add meaningful information to most text processing tasks.

7. Lowercasing:

    Converted all tokens to lowercase.

    Justification: Lowercasing ensures consistency by treating words like "Cat" and "cat" as identical.

8. URL and Email Removal:

    Excluded tokens that represented URLs or emails using like_url and like_email attributes.


    Justification: URLs and emails are rarely relevant for semantic text analysis and could skew results if retained.

9. Digit Removal:

    Removed tokens identified as numbers using is_digit.

    Justification: Numbers often do not contribute to semantic understanding in tasks like topic modeling unless they have specific importance.

10. Final Text Column:

    Stored the cleaned and processed text in a new column named text_pp.

    Justification: To preserve the original text and keep the preprocessed text ready for further analysis.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [31]:
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(stop_words="english")
X = vectorizer.fit_transform(df["text_pp"])

for n_topics in [3, 5, 10]:
    print(f"\nNumber of Topics: {n_topics}")
    lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda_model.fit(X)
    print(f"Perplexity: {lda_model.perplexity(X)}")

n_topics = 5  
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_model.fit(X)
document_topics = lda_model.transform(X)


Number of Topics: 3
Perplexity: 4870.548423310948

Number of Topics: 5
Perplexity: 4525.436854474377

Number of Topics: 10
Perplexity: 4459.795163349863


Use n_topics = 5 as the number of topics for the LDA model. Because it provides the best tradeoff between perplexity reduction and interpretability.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [32]:
def print_top_words(model, feature_names, n_top_words=10):
    topic_words = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topic_words[topic_idx] = top_words
        print(f"Topic #{topic_idx}: {', '.join(top_words)}")
    return topic_words

feature_names = vectorizer.get_feature_names_out()

print("Top words for each topic:")
topic_words = print_top_words(lda_model, feature_names)

Top words for each topic:
Topic #0: say, people, armenian, come, armenians, know, turkish, tell, kill, don
Topic #1: file, use, image, program, window, edu, server, available, run, include
Topic #2: god, people, think, know, don, believe, like, say, thing, time
Topic #3: gun, file, year, weapon, firearm, output, law, time, crime, control
Topic #4: game, team, play, year, player, win, good, season, hockey, think


In [33]:
topic_labels = [
    "Armenian-Turkish Conflict",
    "Technology and Software",
    "Religion and Belief Systems",
    "Gun Control and Crime",
    "Sports games"
]
print("Topic Labels:")
for idx, label in enumerate(topic_labels):
    print(f"Topic #{idx}: {label}")

Topic Labels:
Topic #0: Armenian-Turkish Conflict
Topic #1: Technology and Software
Topic #2: Religion and Belief Systems
Topic #3: Gun Control and Crime
Topic #4: Sports games


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [34]:
document_topics_df = pd.DataFrame(document_topics, columns=[f"Topic #{i}" for i in range(lda_model.n_components)])

document_topics_df["Dominant Topic"] = document_topics_df.idxmax(axis=1)

first_five_documents = document_topics_df.iloc[:5]
print("Topic distribution for the first 5 documents:")
print(first_five_documents)

Topic distribution for the first 5 documents:
   Topic #0  Topic #1  Topic #2  Topic #3  Topic #4 Dominant Topic
0  0.003716  0.003681  0.747783  0.241152  0.003668       Topic #2
1  0.002765  0.988893  0.002799  0.002766  0.002777       Topic #1
2  0.003306  0.353100  0.637065  0.003259  0.003271       Topic #2
3  0.005741  0.597561  0.385503  0.005607  0.005588       Topic #1
4  0.981898  0.004519  0.004582  0.004508  0.004492       Topic #0


Comments on Document Topic Assignment:
1. Document 0 is strongly associated with Topic #2, with a probability of 0.747783, indicating a clear alignment with this topic.
2. Document 1 has a very high probability (0.988083) for Topic #1, which shows a strong and unambiguous relationship with this topic.
3. Document 2 also aligns with Topic #2, with a probability of 0.637605, showing that this document shares content similar to Document 0.
4. Documents 3 and 4 are both assigned to Topic #0. Document 4, in particular, has a very high probability of 0.981898 for Topic #0, suggesting a strong focus on this topic.
5. The model’s topic assignments are mostly distinct, with clear dominant topics for each document. However, the overlap of Documents 3 and 4 with Topic #0 might indicate shared content or terms across similar categories.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-based filtering recommends items to users by analyzing the characteristics of items they have previously interacted with. It builds a profile of user preferences based on features such as keywords, genres, or item descriptions and matches these features to new items.
2. Filter Bubble Effect: Recommender systems often reinforce existing preferences by only suggesting content similar to what users have previously interacted with. This can limit exposure to diverse perspectives, leading to a filter bubble where users are isolated from differing opinions or new experiences.

    Privacy Concerns: Recommender systems require large amounts of user data, such as browsing history, preferences, or demographic information. The collection and use of this data raise privacy issues, especially if it is misused, shared without consent, or breached by malicious actors.

3. Transfer learning in natural language processing involves leveraging a pre-trained model, typically trained on a large corpus like Wikipedia or Common Crawl for a specific task such as sentiment analysis or question answering. Instead of training a model from scratch, transfer learning fine-tunes the pre-trained model on a smaller, task-specific dataset.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)